In [ ]:
import os
import librosa
import math
import json

DATASET_PATH = "Assorted Channels"
JSON_PATH = "data1.json"
SAMPLE_RATE = 22050

## NEED TO UPDATE TO BE DYNAMIC
DURATION = 30 # measured in seconds, but wont work for us
##

SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    # build a dictionary to store data
    data = {
        "mapping": [], #"mapping": ["clacky","thocky"], # 0 = clacky, 1 = thocky
        "mfcc": [], # training inputs
        "labels": [] # target outputs
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # 1.2 -> 2
    
    print('test0')
    # loop through all genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        print("test1")
        print('\n dirpath: {},\n dirnames: {},\n filenames: {}'.format(dirpath,dirnames,filenames))
        # ensure that we are not at the root level
        if dirpath is dataset_path:
            print("test2")
            # save the semantic
            dirpath_components = dirpath.split("/") # genre/blues ==> ["genre", "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\n Processing {}".format(semantic_label))
            
            # process files for a specific genre
            for f in filenames:
                
                # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                
                # process segments extracting mfcc and storing data
                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s 
                    finish_sample = start_sample + num_samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                                sr=sr,
                                                n_fft=n_fft,
                                                n_mfcc=n_mfcc,
                                                hop_length=hop_length
                                               )
                    
                    mfcc = mfcc.T 
                    # store mfcc for segment only if it has the expected length
                    if len(mfcc) != expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, s))                  

                        
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

test0
test1

 dirpath: Assorted Channels,
 dirnames: [],
 filenames: ['7V but sounds like Jelly Epoch - Typing Test.wav', 'Acrylic Alice w Lubed Gateron Yellows.wav', 'Acrylic Arisu by Kibodo Studio with Cherry Hyperglide Browns.wav', 'Acrylic Tofu65 w lubed Holy Pandas.wav', 'Acrylic XO with Lubed Durock L4 Switches Typing Sounds (GMK Dots, POM Jelly Keycaps).wav', 'Ai03 Vega 65_ keyboard typing soundTTC Panda switches FR4 plate.wav', 'Ai03 Vega w Broken-In NovelKeys Cream Sound Test.wav', 'Ai03 Vega w Cherry MX Black (Hyperglides).wav', 'AI03 Vega with Black Cherry Pie Switches Sound Test.wav', 'Bakeneko 60 by CannonKeys _ Lubed Gateron Black Inks v2 Typing Sound Test ASMR.wav', 'Bakeneko 65 V2  Durock Alpaca  Brass Plate  Typing Sounds.wav', 'Bakeneko60 with Alpacas Typing Sounds.wav', 'Black Polycarbonate Think6.5 with lubed Cherry MX Browns Typing Sounds ASMR.wav', 'BOK.CP with lubed Healios Typing Sounds.wav', 'Brutal60 SE with lubed Gateron Black Inks.wav', 'Brutal60 w Lubed Akk